## Getting a bit more deeper into Agents

### Conversational ReAct Agents

This agent is designed for use in conversational settings. It incorporate the React framework to determine which tool to use and utilizes memory to remember previous conversation interactions.

In [ ]:
from langchain.agents import create_react_agent, load_tools
from langchain_google_genai import GoogleGenerativeAI
from langchain.memory import ConversationBufferMemory
from dotenv import load_dotenv
from langchain.agents import initialize_agent
import os
load_dotenv()

In [41]:
llm = GoogleGenerativeAI(
  model="gemini-1.5-pro-latest", 
  google_api_key=os.getenv("GOOGLE_API_KEY"), 
)

In [42]:
tools = load_tools(["llm-math"], llm=llm)

In [44]:
memory = ConversationBufferMemory(memory_key="chat_history")

conversational_agent = initialize_agent(
    agent="conversational-react-description", 
    tools=tools, 
    llm=llm,
    verbose=True,
    max_iterations=3,
    memory=memory,)

In [ ]:
output_1 = conversational_agent.run("Add 7 to 9 and tell me the result")
output_2 = conversational_agent.run("Add 5 to the previous result")

In [ ]:
print(output_1)
print(output_2)

### ReAct Docstore

This agent utilizes the React framewor to communicate with a docstore. It requires the availability of a Search tool and a Lookup tool with identical names. The Search tools is used to search for a document, while the Lookup tool looks up a term within the most recently found document.

A **docstore** in LangChain is essentially a storage system for documents. It provides a way for LangChain agents to access and interact with information. Think of it like a library, but instead of books, it holds pieces of text and associated data.

In [26]:
from langchain.agents import initialize_agent, Tool
from langchain import Wikipedia
from langchain.agents.react.base import DocstoreExplorer

In [27]:
docstore = DocstoreExplorer(Wikipedia())
tools=[
  Tool(name="Search", func=docstore.search, description="useful for when you need to search wikipedia"),
  Tool(name="Lookup", func=docstore.lookup, description="useful for when you need to lookup a term in wikipedia")
]

docstore_agent=initialize_agent(
    tools,
    llm,
    agent="react-docstore",
    verbose=True,
    handle_parsing_errors=True,
    max_iterations=5
)

In [ ]:
docstore_agent("Tell me a few key things about Ho Chi Minh.")

### Self-ask with Search
This agent utilizes the intermediate Answer tool for self-asking questions.

In [17]:
from langchain.tools import DuckDuckGoSearchRun
search = DuckDuckGoSearchRun()

tools = [
  Tool(
    func=search.run,
    name="Intermediate Answer",
    description="Useful for when you need to search the Internet for information"
  )
]

agent = initialize_agent(
  tools=tools,
  llm=llm,
  agent="self-ask-with-search",
  verbose=True,
  handle_parsing_errors=True,
  max_interations=3
)

In [ ]:
import time

# Add a delay to avoid hitting the rate limit
time.sleep(2)
print(agent.invoke("Question: Who was the president of USA when the first Moon landing took place?"))